In [1]:
#! /usr/bin/env python
# 06/28/2022 Zheting
from pymatgen.core import Lattice, Structure, Molecule
import random
import numpy as np

In [2]:
# Read a POSCAR and write to a CIF.
structure = Structure.from_file("/home/zheting/data/vasp/supercell/addFe_lowsym/addFeO/random/1/POSCAR")
# structure.to(filename="CsCl.cif")

# Read an xyz file and write to a Gaussian Input file.
# methane = Molecule.from_file("methane.xyz")
# methane.to(filename="methane.gjf")
print(structure)


Full Formula (Sr8 Ca4 Fe1 Cu8 Bi8 O33)
Reduced Formula: Sr8Ca4FeCu8Bi8O33
abc   :   5.454364   5.455064  46.876252
angles:  89.881191  90.089505  90.017961
Sites (62)
  #  SP           a         b         c  selective_dynamics
---  ----  --------  --------  --------  ---------------------
  0  Sr    0.512701  0.006254  0.241164  [False, False, False]
  1  Sr    0.004807  0.507248  0.241197  [False, False, False]
  2  Sr    0.005263  0.014191  0.438839  [False, False, False]
  3  Sr    0.505545  0.506305  0.438833  [False, False, False]
  4  Sr    0.010175  0.01529   0.581336  [True, True, True]
  5  Sr    0.506304  0.503154  0.581996  [True, True, True]
  6  Sr    0.509815  0.01109   0.098813  [False, False, False]
  7  Sr    0.001444  0.511295  0.098831  [False, False, False]
  8  Ca    0.007911  0.008784  0.510033  [False, False, False]
  9  Ca    0.507799  0.507268  0.510015  [False, False, False]
 10  Ca    0.507783  0.00953   0.169981  [False, False, False]
 11  Ca    0.006178  0.

In [3]:
# Get the distance between position1 and position2 in Cartesian coordinates
# Usage: 
# position1 and position2 are 3d-vector indicating two positions, either in fractional or Cartesian coordinates
# When using fractional coordinates, cubic lattice is ASSUMED: lattice = [a, b, c]
# When using Cartesian coordinates, lattice = [1, 1, 1] or "Cartesian", only the first letter matters
def get_bond_length(position1, position2, lattice):
    if lattice[0]=="C" or lattice[0]=="c":
        lattice = np.array([1,1,1])
    else:
        lattice = np.array(lattice)
    position1 = np.array(position1)
    position2 = np.array(position2)
    # get the difference
    diff = abs(position1 - position2)
    # consider the periodic boundary condition
    diff = np.minimum(diff, 1.0 - diff)
    # get the difference in Cartesian
    diff = diff * lattice
    # get the bond length
    bond = np.linalg.norm(diff)**2
    return bond

# Randomly generate an atomic position
# Usage:
# pos_range is the position range like [[xmin, xmax], [ymin, ymax], ...]
def gen_random_atom(pos_range):
    pos_atom = []
    for axis_range in pos_range:
        pos_atom.append(random.uniform(axis_range[0], axis_range[1]))
    return pos_atom

# Determine if we can accept the new atomic position (too close to other atoms?)
# Usage: 
# new_atom and old_atoms are 3d-vector(s) for new atomic position and old atomic positions
# When using fractional coordinates, cubic lattice is ASSUMED: lattice = [a, b, c]
# When using Cartesian coordinates, lattice = [1, 1, 1] or "Cartesian", only the first letter matters
# tol is the smallest tolerance distance between two atoms in Cartesian coordinates
def atom_isgood(new_atom, old_atoms, lattice, tol):
    isgood = True
    for iatom in old_atoms:
        ibond = get_bond_length(new_atom, iatom, lattice)
        isgood = isgood and (ibond > tol)
    return isgood


In [8]:
atoms_label = [24, 25, 50, 51, 60, 61]
atoms_element = ["Bi", "Bi", "O", "O", "O", "Fe"]
num_atoms = len(atoms_label)
lattice = [5.454364, 5.455064, 46.876252]
pos_range = [[0, 1], [0, 1], [0.62, 0.7]]
old_atoms = []
tol = 0.8
num_structures = 30

# Generate several atoms randomly in a given range. The distance between two atoms are larger than a smallest tolerance
# label of the structure
istruct = 1
count = 0
while len(old_atoms) < num_atoms:
    pos_atom = []
    for axis_range in pos_range:
        pos_atom.append(random.uniform(axis_range[0], axis_range[1]))
    if atom_isgood(pos_atom, old_atoms, lattice, tol):
        old_atoms.append(pos_atom)
    if count > 100:
        print("Impossible to find good atom position")
        break

# Insert the generated atoms in the structure
new_struct = structure
for iat in range(num_atoms):
    atom_label = atoms_label[iat]
    new_struct[atom_label] = atoms_element[iat], old_atoms[iat]

# save structure
namestruct = "/home/zheting/data/vasp/supercell/addFe_lowsym/addFeO/random/random_struct/POSCAR_" + str(istruct)
new_struct.to(filename=namestruct)

'POS3'